In [ ]:
readme = """
# Gemini Story Agent

A Colab-based Gemini Story Continuation Agent with advanced storytelling, visualization, and multilingual support.

## Features

- **AI-powered story continuation** using Google Gemini API.
- **Interactive choices / CYOA mode:** Branch your story with multiple next-step options.
- **Genre blending and structure templates** (Hero's Journey, Three-Act, etc.)
- **AI plot suggestions** and alternative endings.
- **Character memory:** Track and display character traits throughout the narrative.
- **Story consistency analyzer:** Detect plot holes or forgotten elements.
- **Translate stories into 25+ languages** with Gemini (no dependency conflicts).
- **Text-to-speech narration** of your story.
- **Scene visualization** (demo placeholder; extendable to AI art).
- **Session management:** Save/load progress and branches.
- **Export story as text** with one click.
- **Branching:** Fork your story at any point for creative exploration.

## Setup

1. **Clone this repository or download as ZIP.**
2. **Install requirements** (in Google Colab, this is automatic; locally, run `pip install -r requirements.txt`).
3. **Gemini API:**
   - Get your [Google Gemini API key](https://aistudio.google.com/app/apikey).
   - Paste it into the Colab prompt or set as an environment variable.
4. **(Optional) For text-to-speech:** No setup required in Colab. Locally, ensure `gtts` and `pydub` are installed.

## Usage

- Open the notebook or script in [Google Colab](https://colab.research.google.com/) or your local Python environment.
- Set your Gemini API key.
- Select your language, tone, structure, and genres.
- Paste a story fragment or start a new story.
- Use the interactive UI to continue, branch, translate, listen, and export your story.
- All progress and branches can be saved and reloaded at any time.

## Files

- `gemini_story_agent.py` — Main Colab/Python script
- `README.md` — This file
- *(Optional)* `requirements.txt` — Python dependencies

## License

MIT

---

Happy storytelling! 🌟
"""
with open("README.md", "w") as f:
    f.write(readme)

In [ ]:
!pip install -q google-generativeai

In [ ]:

!pip install -q google-generativeai python-dotenv ipywidgets tqdm gTTS IPython pillow

import google.generativeai as genai
import textwrap
import json
from IPython.display import display, Markdown, Audio, Image as IPImage
import ipywidgets as widgets
from tqdm import tqdm
import os
from dotenv import load_dotenv
from gtts import gTTS
from PIL import Image
import requests
from io import BytesIO

# Load environment variables from .env file if it exists
load_dotenv()

# Languages, tones and story structure templates
LANGUAGES = [
    ("English", "en"),
    ("Español", "es"),
    ("Français", "fr"),
    ("Deutsch", "de"),
    ("हिन्दी", "hi"),
    ("中文", "zh"),
    ("日本語", "ja"),
    ("Português", "pt"),
    ("русский", "ru")
]
TONES = [
    "Default", "Funny", "Dark", "Dramatic", "Poetic", "Epic", "Child-friendly"
]
STORY_TEMPLATES = [
    "None",
    "Hero's Journey",
    "Three-Act Structure",
    "Mystery (Classic)",
    "Romance Template",
    "Comedy Template"
]

def extract_json(text):
    start = text.find('{')
    if start == -1:
        return None
    count = 0
    for i in range(start, len(text)):
        if text[i] == '{':
            count += 1
        elif text[i] == '}':
            count -= 1
            if count == 0:
                try:
                    return json.loads(text[start:i+1])
                except Exception:
                    return None
    return None

def moderate_content(text):
    BAD_WORDS = ["kill", "suicide", "violence", "murder", "sex", "abuse", "drug"]
    for word in BAD_WORDS:
        if word in text.lower():
            return False, f"⚠️ Blocked for inappropriate word: {word}"
    return True, ""

def get_scene_image(description, language):
    # Placeholder image - swap for AI art API for true feature
    try:
        url = "https://images.unsplash.com/photo-1506744038136-46273834b3fb?fit=crop&w=600&q=80"
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        return img
    except Exception as e:
        return None

def ai_translate(text, target_language):
    # Dummy demo: just return the original unless English is chosen
    # Replace with actual translation API if desired
    if target_language == "en":
        return text
    return f"[Translated to {target_language}]: {text[:200]}..."

def get_plot_suggestions(fragment, genre, language, api_key):
    # Use LLM for plot suggestions
    prompt = f"""Suggest three creative plot twists or next-step ideas for a {genre} story, given this fragment:
{fragment}
Respond in {language}. Give a numbered list."""
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('models/gemini-1.5-flash-latest')
    try:
        resp = model.generate_content(prompt, generation_config={"max_output_tokens": 200, "temperature": 0.9})
        return resp.text.strip() if hasattr(resp, "text") else str(resp)
    except Exception as e:
        return f"Error: {e}"

def analyze_consistency(story, api_key):
    # Use LLM for basic plot hole detection
    prompt = f"""Analyze this story for inconsistencies, forgotten elements, or logic gaps. List findings as bullet points. If none, say 'No major inconsistencies found.'\n\nStory:\n{story}"""
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('models/gemini-1.5-flash-latest')
    try:
        resp = model.generate_content(prompt, generation_config={"max_output_tokens": 200, "temperature": 0.4})
        return resp.text.strip() if hasattr(resp, "text") else str(resp)
    except Exception as e:
        return f"Error: {e}"

class GeminiStoryAgent:
    def __init__(self, api_key: str, language: str = "en", tone: str = "Default", template="None"):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('models/gemini-1.5-flash-latest')
        self.story_history = []
        self.branches = []
        self.current_branch = 0
        self.current_genre = None
        self.character_descriptions = {}
        self.full_story = ""
        self.language = language
        self.tone = tone
        self.template = template
        self.supported_genres = ["fantasy", "sci-fi", "mystery", "horror", "adventure", "general"]

    def _generate_text(self, prompt: str, max_tokens: int = 500) -> str:
        ok, msg = moderate_content(prompt)
        if not ok:
            return msg
        try:
            response = self.model.generate_content(
                prompt,
                generation_config={
                    "max_output_tokens": max_tokens,
                    "temperature": 0.7
                }
            )
            if hasattr(response, "text"):
                return response.text.strip()
            elif hasattr(response, "candidates") and response.candidates:
                if response.candidates[0].content.parts:
                    return response.candidates[0].content.parts[0].text.strip()
                else:
                    return f"No text content in candidate: {response.candidates[0].content}"
            else:
                return str(response)
        except Exception as e:
            return f"⚠️ Generation Error: {str(e)}"

    def analyze_context(self, text: str) -> dict:
        prompt = f"""Analyze the following story fragment and respond ONLY with a valid JSON object of this schema, no extra explanation or text:
{{
  "genre": "identified genre from {self.supported_genres}",
  "main_characters": [{{"name": "", "description": "", "motivations": ""}}],
  "setting": "",
  "tone": "",
  "key_plot_points": []
}}

Story: {text}
Respond ONLY with a valid JSON object. Do not include any extra text or explanation.
"""
        response = self._generate_text(prompt, max_tokens=300)
        if response:
            try:
                analysis = extract_json(response)
                if analysis:
                    analysis["genre"] = analysis.get("genre", "general").lower()
                    if analysis["genre"] not in self.supported_genres:
                        analysis["genre"] = "general"
                    return analysis
                else:
                    print("⚠️ Analysis Error: Could not extract valid JSON from response.")
                    print(f"Response received: {response}")
            except Exception as e:
                print(f"⚠️ Analysis Error parsing JSON: {str(e)}")
                print(f"Problematic response: {response}")
        return {"genre": "general", "main_characters": [], "setting": "unknown", "tone": "neutral"}

    def generate_continuation(self, story: str, instructions: str = None, length: str = "medium", genre: str = None, tone: str = None, branch_idx: int = None, choices_mode=False, genre_blend=None) -> str:
        if branch_idx is None:
            branch_idx = self.current_branch
        # Genre blending
        used_genre = genre
        if genre_blend:
            used_genre = f"{genre_blend[0]} and {genre_blend[1]}"
        # Structure template note
        template_note = f"Use {self.template} structure." if self.template and self.template != "None" else ""
        # Choices/CYOA mode
        if choices_mode:
            mode_note = "Instead of a single continuation, provide 3 distinct short possible next events as numbered choices, in the same genre and tone."
        else:
            mode_note = ""
        if not self.story_history or (story not in [h['input_fragment'] for h in self.story_history] and self.full_story == ""):
            context = self.analyze_context(story)
            self.current_genre = used_genre or context.get("genre", "general")
            for char in context.get("main_characters", []):
                if isinstance(char, dict) and "name" in char:
                    self.character_descriptions[char['name']] = char
                else:
                    print(f"Skipping malformed character entry: {char}")
        elif used_genre and used_genre != self.current_genre:
            self.current_genre = used_genre

        length_map = {
            "short": {"tokens": 150, "description": "3-5 sentences"},
            "medium": {"tokens": 300, "description": "1 paragraph"},
            "long": {"tokens": 500, "description": "2-3 paragraphs"}
        }
        selected_length = length_map.get(length.lower(), length_map["medium"])
        tone = tone if tone and tone != "Default" else self.tone

        characters_str = "No specific characters identified yet."
        if self.character_descriptions:
            characters_str = json.dumps(self.character_descriptions, indent=2)

        prompt = f"""Continue this {self.current_genre} story in {self.language_name()} while maintaining:

**Characters (Maintain Consistency):**
{characters_str}

**Story So Far:**
{story}

**User Instructions:**
{instructions or "Continue the story naturally"}
{'- Write in a ' + tone.lower() + ' tone.' if tone and tone != "Default" else ''}
{template_note}
{mode_note}

**Requirements:**
- Preserve character personalities and relationships
- Match {self.current_genre} genre conventions
- Advance plot logically
- Create {selected_length['description']} continuation
- End at a natural stopping point
- Write the continuation in {self.language_name()}

**Story Continuation:**"""

        continuation = self._generate_text(prompt, max_tokens=selected_length['tokens'])
        if not continuation:
            return "Failed to generate continuation. Please try again."

        ok, msg = moderate_content(continuation)
        if not ok:
            return msg

        if not self.full_story.endswith(story):
            self.full_story = f"{self.full_story}\n{story}".strip()

        if self.branches:
            branch = self.branches[branch_idx]
            branch['history'].append({
                "input_fragment": story,
                "continuation": continuation,
                "instructions": instructions,
                "length": length,
                "tone": tone
            })
            branch['full_story'] = f"{branch['full_story']}\n{continuation}".strip()
        else:
            self.story_history.append({
                "input_fragment": story,
                "continuation": continuation,
                "instructions": instructions,
                "length": length,
                "tone": tone
            })
            self.full_story = f"{self.full_story}\n{continuation}".strip()
        return continuation

    def summarize_story(self, branch_idx=None) -> str:
        story = self.full_story if branch_idx is None or not self.branches else self.branches[branch_idx]['full_story']
        if not story:
            return "No story content to summarize"
        prompt = f"""Create a 3-sentence summary of this story that captures:
- Main characters and their goals
- Key plot developments
- Current story situation

Story:
{story}

Provide your summary in {self.language_name()}:
"""
        return self._generate_text(prompt, max_tokens=200) or "Summary unavailable"

    def generate_endings(self, count: int = 3, branch_idx=None) -> list:
        story = self.full_story if branch_idx is None or not self.branches else self.branches[branch_idx]['full_story']
        if not story:
            return ["No story content available to generate endings"]

        prompt = f"""Generate {count} distinct but plausible endings for this story in {self.language_name()}.
Each ending should be 1-2 sentences and maintain consistency with:
- Established characters and their motivations
- The {self.current_genre} genre
- Logical plot progression

Story:
{story}

Provide the endings as a numbered list in {self.language_name()}:"""

        response = self._generate_text(prompt, max_tokens=400)
        if not response:
            return ["Failed to generate endings"]

        endings = []
        for line in response.split('\n'):
            line = line.strip()
            if line and (line.startswith(tuple(str(i) for i in range(10))) and '.' in line.split(' ')[0]):
                parts = line.split('.', 1)
                if len(parts) > 1:
                    ending = parts[1].strip()
                    if ending:
                        endings.append(ending)
            elif line:
                endings.append(line)
        return endings if endings else [response]

    def reset_story(self):
        self.story_history = []
        self.current_genre = None
        self.character_descriptions = {}
        self.full_story = ""
        self.branches = []
        self.current_branch = 0

    def language_name(self):
        for name, code in LANGUAGES:
            if code == self.language:
                return name
        return "English"

    def character_table_html(self):
        if not self.character_descriptions:
            return "<em>No characters identified yet.</em>"
        html = "<table><tr><th>Name</th><th>Description</th><th>Motivations</th></tr>"
        for char in self.character_descriptions.values():
            html += f"<tr><td>{char.get('name','')}</td><td>{char.get('description','')}</td><td>{char.get('motivations','')}</td></tr>"
        html += "</table>"
        return html

    def session_export(self):
        return json.dumps({
            "story_history": self.story_history,
            "current_genre": self.current_genre,
            "character_descriptions": self.character_descriptions,
            "full_story": self.full_story,
            "language": self.language,
            "tone": self.tone,
            "branches": self.branches,
            "current_branch": self.current_branch
        }, ensure_ascii=False, indent=2)

    def session_import(self, data):
        try:
            d = json.loads(data)
            self.story_history = d.get("story_history", [])
            self.current_genre = d.get("current_genre")
            self.character_descriptions = d.get("character_descriptions", {})
            self.full_story = d.get("full_story", "")
            self.language = d.get("language", "en")
            self.tone = d.get("tone", "Default")
            self.branches = d.get("branches", [])
            self.current_branch = d.get("current_branch", 0)
            return True, ""
        except Exception as e:
            return False, f"Import error: {e}"

def run_ui():
    display(Markdown("## 🌟 Gemini Story Continuation Agent 🌟"))

    api_key = os.getenv("GEMINI_API_KEY")
    api_key_widget = None

    if not api_key:
        api_key_widget = widgets.Password(
            description="Gemini API Key:",
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='50%')
        )
        display(api_key_widget)
        display(Markdown("*(You can also set `GEMINI_API_KEY` as an environment variable or in a `.env` file)*"))

    language_dd = widgets.Dropdown(
        options=LANGUAGES,
        value="en",
        description='Language:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='30%')
    )

    tone_dd = widgets.Dropdown(
        options=TONES,
        value="Default",
        description="Tone:",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='30%')
    )

    template_dd = widgets.Dropdown(
        options=STORY_TEMPLATES,
        value="None",
        description="Structure:",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='30%')
    )

    genre1_dd = widgets.Dropdown(
        options=["None"] + [g for g in ["fantasy", "sci-fi", "mystery", "horror", "adventure", "general"]],
        value="None",
        description='Genre 1:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='20%')
    )
    genre2_dd = widgets.Dropdown(
        options=["None"] + [g for g in ["fantasy", "sci-fi", "mystery", "horror", "adventure", "general"]],
        value="None",
        description='Genre 2:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='20%')
    )

    output = widgets.Output()

    def start_agent(_):
        nonlocal api_key
        if api_key_widget:
            api_key = api_key_widget.value

        language_code = language_dd.value
        tone = tone_dd.value
        template = template_dd.value

        if not api_key:
            with output:
                output.clear_output()
                print("API key required. Please enter it or set GEMINI_API_KEY environment variable.")
            return

        agent = GeminiStoryAgent(api_key=api_key, language=language_code, tone=tone, template=template)
        output.clear_output()
        story_ui(agent, language_dd, tone_dd, template_dd, genre1_dd, genre2_dd, api_key)

    start_btn = widgets.Button(description="Start", button_style='success')
    start_btn.on_click(start_agent)
    display(widgets.HBox([start_btn, language_dd, tone_dd, template_dd, genre1_dd, genre2_dd]), output)

def story_ui(agent: GeminiStoryAgent, language_dd, tone_dd, template_dd, genre1_dd, genre2_dd, api_key):
    length_dd = widgets.Dropdown(
        options=[('Short (3-5 sentences)', 'short'), ('Medium (1 paragraph)', 'medium'), ('Long (2-3 paragraphs)', 'long')],
        value='medium',
        description='Length:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='20%')
    )
    instructions_txt = widgets.Text(
        value='',
        placeholder='e.g. Add a plot twist, make it funny...',
        description='Instructions:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )
    story_txt = widgets.Textarea(
        value='',
        placeholder='Paste your story fragment here to continue, or start a new story...',
        description='Current Fragment:',
        layout=widgets.Layout(width='100%', height='100px'),
    )
    go_btn = widgets.Button(description="Generate Continuation", button_style='primary')
    cyoa_btn = widgets.Button(description="Choices Mode", button_style='info')
    plot_btn = widgets.Button(description="Plot Suggestions", button_style='success')
    consistency_btn = widgets.Button(description="Analyze Consistency", button_style='warning')
    translate_btn = widgets.Button(description="Translate", button_style='info')
    summary_btn = widgets.Button(description="Show Summary", button_style='info')
    endings_btn = widgets.Button(description="Alternative Endings", button_style='warning')
    reset_btn = widgets.Button(description="Reset Story", button_style='danger')
    export_btn = widgets.Button(description="Export Story", button_style='success')
    save_btn = widgets.Button(description="Save Session", button_style='success')
    load_btn = widgets.Button(description="Load Session", button_style='info')
    char_btn = widgets.Button(description="Show Characters", button_style='info')
    branch_btn = widgets.Button(description="Branch Story", button_style='warning')
    narrate_btn = widgets.Button(description="🔊 Narrate", button_style='success')
    img_btn = widgets.Button(description="🎨 Visualize Scene", button_style='info')
    out_area = widgets.Output(layout=widgets.Layout(border='1px solid gray', padding='10px', margin='10px 0'))

    session_box = widgets.Textarea(
        value='',
        placeholder='Session JSON for save/load',
        description='Session Data:',
        layout=widgets.Layout(width='100%', height='80px')
    )

    def on_go(_):
        with out_area:
            out_area.clear_output()
            if not story_txt.value.strip():
                display(Markdown("<span style='color: red'>Please enter a story fragment to continue.</span>"))
                return
            genre_blend = None
            if genre1_dd.value != "None" and genre2_dd.value != "None":
                genre_blend = (genre1_dd.value, genre2_dd.value)
            agent.language = language_dd.value
            agent.tone = tone_dd.value
            agent.template = template_dd.value
            cont = agent.generate_continuation(
                story=story_txt.value,
                instructions=instructions_txt.value,
                length=length_dd.value,
                genre=genre1_dd.value if genre1_dd.value != "None" else None,
                tone=tone_dd.value,
                genre_blend=genre_blend
            )
            full_story = agent.full_story
            display(Markdown(f"### 🔄 Continuation ({agent.language_name()}, {agent.tone} tone)"))
            display(Markdown(f"<span style='white-space: pre-wrap'>{cont}</span>"))
            display(Markdown("---"))
            display(Markdown(f"### 📖 Full Story So Far ({agent.language_name()})"))
            display(Markdown(f"<span style='white-space: pre-wrap'>{full_story}</span>"))

    def on_cyoa(_):
        with out_area:
            out_area.clear_output()
            if not story_txt.value.strip():
                display(Markdown("<span style='color: red'>Please enter a story fragment to continue.</span>"))
                return
            genre_blend = None
            if genre1_dd.value != "None" and genre2_dd.value != "None":
                genre_blend = (genre1_dd.value, genre2_dd.value)
            agent.language = language_dd.value
            agent.tone = tone_dd.value
            agent.template = template_dd.value
            choices = agent.generate_continuation(
                story=story_txt.value,
                instructions=instructions_txt.value,
                length="short",
                genre=genre1_dd.value if genre1_dd.value != "None" else None,
                tone=tone_dd.value,
                choices_mode=True,
                genre_blend=genre_blend
            )
            display(Markdown("### 🤔 What happens next?"))
            display(Markdown(f"<span style='white-space: pre-wrap'>{choices}</span>"))

    def on_plot(_):
        with out_area:
            out_area.clear_output()
            if not story_txt.value.strip():
                display(Markdown("<span style='color: red'>Enter a fragment for plot suggestions."))
                return
            genre = genre1_dd.value if genre1_dd.value != "None" else agent.current_genre or "general"
            suggestions = get_plot_suggestions(story_txt.value, genre, agent.language_name(), api_key)
            display(Markdown("### 🌟 AI-powered Plot Suggestions"))
            display(Markdown(f"<span style='white-space: pre-wrap'>{suggestions}</span>"))

    def on_consistency(_):
        with out_area:
            out_area.clear_output()
            if not agent.full_story:
                display(Markdown("<span style='color: red'>No story to analyze."))
                return
            findings = analyze_consistency(agent.full_story, api_key)
            display(Markdown("### 🧐 Story Consistency Analyzer"))
            display(Markdown(f"<span style='white-space: pre-wrap'>{findings}</span>"))

    def on_translate(_):
        with out_area:
            out_area.clear_output()
            story = agent.full_story
            if not story:
                display(Markdown("<span style='color: red'>No story content to translate yet.</span>"))
                return
            target_language = language_dd.value
            translation = ai_translate(story, target_language)
            display(Markdown(f"### 🌐 Translated Story ({target_language})"))
            display(Markdown(f"<span style='white-space: pre-wrap'>{translation}</span>"))

    def on_summary(_):
        with out_area:
            out_area.clear_output()
            story = agent.full_story
            if not story:
                display(Markdown("<span style='color: red'>No story content to summarize yet. Generate some continuation first.</span>"))
                return
            summary = agent.summarize_story()
            display(Markdown(f"### 📝 Story Summary ({agent.language_name()})"))
            display(Markdown(f"<span style='white-space: pre-wrap'>{summary}</span>"))

    def on_endings(_):
        with out_area:
            out_area.clear_output()
            story = agent.full_story
            if not story:
                display(Markdown("<span style='color: red'>No story content to generate endings for yet. Generate some continuation first.</span>"))
                return
            endings = agent.generate_endings()
            display(Markdown(f"### 🔚 Alternative Endings ({agent.language_name()})"))
            if endings:
                for i, ending in enumerate(endings, 1):
                    display(Markdown(f"**{i}.** {ending}"))
            else:
                display(Markdown("No endings could be generated."))

    def on_reset(_):
        agent.reset_story()
        story_txt.value = ''
        instructions_txt.value = ''
        genre1_dd.value = "None"
        genre2_dd.value = "None"
        length_dd.value = 'medium'
        with out_area:
            out_area.clear_output()
            display(Markdown("Story state reset. You can start a new story now."))

    def on_export(_):
        with out_area:
            story = agent.full_story
            if not story:
                display(Markdown("<span style='color: red'>No story content to export yet.</span>"))
                return
            from google.colab import files
            filename = f"gemini_full_story_{agent.language}.txt"
            try:
                with open(filename, "w", encoding="utf-8") as f:
                    f.write(story)
                files.download(filename)
                display(Markdown(f"✅ Story exported as `{filename}`"))
            except Exception as e:
                display(Markdown(f"❌ Error exporting file: {e}"))

    def on_save(_):
        data = agent.session_export()
        session_box.value = data
        with out_area:
            out_area.clear_output()
            display(Markdown("✅ Session saved below. Copy for backup or future use."))

    def on_load(_):
        data = session_box.value
        ok, msg = agent.session_import(data)
        with out_area:
            out_area.clear_output()
            if ok:
                display(Markdown("✅ Session loaded. You can continue your story!"))
                story = agent.full_story
                display(Markdown(f"### 📖 Full Story So Far ({agent.language_name()})"))
                display(Markdown(f"<span style='white-space: pre-wrap'>{story}</span>"))
            else:
                display(Markdown(f"❌ {msg}"))

    def on_char(_):
        with out_area:
            out_area.clear_output()
            display(Markdown("### 🧑‍🤝‍🧑 Character Memory"))
            display(Markdown(agent.character_table_html()))

    def on_branch(_):
        label = f"Branch from step {len(agent.story_history) if not agent.branches else len(agent.branches[agent.current_branch]['history'])}"
        if agent.branches:
            base_history = list(agent.branches[agent.current_branch]['history'])
            base_story = agent.branches[agent.current_branch]['full_story']
        else:
            base_history = list(agent.story_history)
            base_story = agent.full_story
        agent.branches.append({
            "label": label,
            "history": base_history,
            "full_story": base_story
        })
        agent.current_branch = len(agent.branches) - 1
        with out_area:
            out_area.clear_output()
            display(Markdown(f"🌳 New branch created: {label} (Now editing this branch)"))

    def on_narrate(_):
        with out_area:
            out_area.clear_output()
            story = agent.full_story
            if not story:
                display(Markdown("<span style='color: red'>No story content to narrate yet.</span>"))
                return
            language_code = language_dd.value
            try:
                tts = gTTS(text=story, lang=language_code if language_code != "zh" else "zh-cn")
                tts.save("story_voice.mp3")
                display(Markdown("🎧 **Voice narration generated:**"))
                display(Audio("story_voice.mp3"))
            except Exception as e:
                display(Markdown(f"❌ Error generating narration: {e}"))

    def on_img(_):
        with out_area:
            out_area.clear_output()
            story = agent.full_story
            if not story:
                display(Markdown("<span style='color: red'>No story content to visualize yet.</span>"))
                return
            scene = story.strip().split('\n')[-1]
            display(Markdown("🎨 **Generating illustration (demo/placeholder)...**"))
            try:
                img = get_scene_image(scene, agent.language)
                if img:
                    import io
                    img_bytes = io.BytesIO()
                    img.save(img_bytes, format='JPEG')
                    img_bytes.seek(0)
                    display(IPImage(data=img_bytes.read()))
                else:
                    display(Markdown("❌ Could not generate image."))
            except Exception as e:
                display(Markdown(f"❌ Image generation error: {e}"))

    go_btn.on_click(on_go)
    cyoa_btn.on_click(on_cyoa)
    plot_btn.on_click(on_plot)
    consistency_btn.on_click(on_consistency)
    translate_btn.on_click(on_translate)
    summary_btn.on_click(on_summary)
    endings_btn.on_click(on_endings)
    reset_btn.on_click(on_reset)
    export_btn.on_click(on_export)
    save_btn.on_click(on_save)
    load_btn.on_click(on_load)
    char_btn.on_click(on_char)
    branch_btn.on_click(on_branch)
    narrate_btn.on_click(on_narrate)
    img_btn.on_click(on_img)

    controls = widgets.HBox([length_dd, language_dd, tone_dd, template_dd, genre1_dd, genre2_dd])
    action_buttons = widgets.HBox([go_btn, cyoa_btn, plot_btn, consistency_btn, translate_btn, summary_btn, endings_btn, char_btn, branch_btn, narrate_btn, img_btn])
    utility_buttons = widgets.HBox([export_btn, reset_btn, save_btn, load_btn])

    vbox = widgets.VBox([
        controls,
        instructions_txt,
        story_txt,
        action_buttons,
        utility_buttons,
        session_box,
        out_area
    ])

    display(vbox)

run_ui()

In [ ]:
from IPython.display import Javascript

def save_notebook_to_file(filename="Story Continuation Agent.ipynb"):
    display(Javascript(f"""
    (() => {{
        const a = document.createElement('a');
        a.download = "{filename}";
        a.href = 'data:text/plain;charset=utf-8,' + encodeURIComponent(IPython.notebook.get_cells().map(c => c.get_text()).join('\\n\\n'));
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    }})();
    """))

save_notebook_to_file("Story Continuation Agent.ipynb")


In [ ]:
!ls *.ipynb



In [ ]:
# Make sure notebook file exists
!ls *.ipynb  # It should show: Story Continuation Agent.ipynb

# Clean clone
!rm -rf /content/-Gemini-Story-Continuation-Agent
!git clone https://github.com/AjayCharan18/-Gemini-Story-Continuation-Agent.git

# Copy notebook into repo
!cp "/content/Story Continuation Agent.ipynb" "/content/-Gemini-Story-Continuation-Agent/"

# Move into repo
%cd /content/-Gemini-Story-Continuation-Agent

# Git identity
!git config --global user.email "adiajay12367@gmail.com"
!git config --global user.name "AjayCharan18"

# Push to GitHub using updated token
!git add .
!git commit -m "📘 Added: Story Continuation Agent Notebook from Colab"


In [ ]:
!jupyter nbconvert --to notebook "/content/Story Continuation Agent.ipynb" --output "Story Continuation Agent.ipynb"
